In [1]:
import pandas as pd
import numpy as np

import os
import zipfile

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score
from sklearn.metrics import f1_score, accuracy_score

In [ ]:
# dirpath = '/home/sergio/Desktop/DataScienceHW/HW5'

# base_path = dirpath + '/homework/data'
# dir_names = os.listdir(base_path)
# dir_names